In [0]:
!pip3 install tensorflow_gpu==2.0


import tensorflow as tf
print(tf.__version__)

2.0.0


In [0]:

!pip3 install -q ktrain

In [0]:
import pandas as pd
import ktrain
from ktrain import text
import numpy as np
from sklearn.model_selection import train_test_split

using Keras version: 2.2.4-tf


In [0]:
data = pd.read_csv('/content/drive/My Drive/Máster KSchool/TFM/data/filtered_df_with_imgNames.csv',usecols=['RATING','DESCRIPTION'])
data = data.apply(lambda x: x.str.replace(',','.'))

In [0]:
data

,DESCRIPTION,RATING
0,When struggling rapper Coco (Azealia Banks) en...,5.3
1,A man taking revenge on the killers of his loy...,6.4
2,The story of a young Viking boy who accompanie...,5.5
3,A dark fairy tale about a gang of five childre...,7.0
4,Four frames of simultaneous action that altern...,6.0
...,...,...
107176,Add a Plot Ã‚Â»,7.3
107177,One of only two survivors from a Martian exped...,5.3
107178,Add a Plot Ã‚Â»,5.1
107179,Is having fun and saying what you really think...,5.5


In [0]:
data['RATING']  = data['RATING'].astype(float)
data['RATING'] = pd.cut( data['RATING'], bins = [0, 5, 6.5, 10], labels=['0','1','2'])
data

,DESCRIPTION,RATING
0,When struggling rapper Coco (Azealia Banks) en...,1
1,A man taking revenge on the killers of his loy...,1
2,The story of a young Viking boy who accompanie...,1
3,A dark fairy tale about a gang of five childre...,2
4,Four frames of simultaneous action that altern...,1
...,...,...
107176,Add a Plot Ã‚Â»,2
107177,One of only two survivors from a Martian exped...,1
107178,Add a Plot Ã‚Â»,1
107179,Is having fun and saying what you really think...,1


In [0]:
print(len(data[data['RATING']=='2']))
print(len(data[data['RATING']=='1']))
print(len(data[data['RATING']=='0']))

40270
47141
19770


In [0]:
MODEL_NAME = 'distilbert-base-uncased'
classes=['0','1','2']
t = text.Transformer(MODEL_NAME, maxlen=500, classes=classes)

In [0]:
model = t.get_model()
model.summary()


Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  2307      
_________________________________________________________________
dropout_19 (Dropout)         multiple                  0         
Total params: 66,955,779
Trainable params: 66,955,779
Non-trainable params: 0
_________________________________________________________________


In [0]:
X = data.DESCRIPTION.values
Y = data.RATING.values

In [0]:
#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
x_train, x_test, y_train, y_test= train_test_split(X, Y.astype(str), stratify=Y, test_size=0.2)

In [0]:
x_train = list(x_train)
x_test = list(x_test)

In [0]:
y_train = np.array(y_train).astype(int)
y_test = np.array(y_test).astype(int)

In [0]:
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 27
	95percentile : 41
	99percentile : 44


preprocessing test...
language: en
test sequence lengths:
	mean : 27
	95percentile : 41
	99percentile : 44


In [0]:
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=128)

In [0]:
### To freeze Distilbert weights
dBert = model.layers[0]
dBert_transformer1 = dBert.transformer.layer[0]
dBert.trainable = False

In [0]:
learner.fit_onecycle(5e-5, 3)



begin training using onecycle policy with max lr of 5e-05...
Train for 670 steps, validate for 168 steps
Epoch 1/3
 97/670 [===>..........................] - ETA: 37:41 - loss: 1.0658 - accuracy: 0.4318

In [0]:
learner.validate(class_names=t.get_classes())